In [8]:
library(tidyverse)
library(tidymodels)

ERROR: Error in library(tidyext): there is no package called ‘tidyext’


# **Project Proposal for DSCI 100: Predicting Usage of a Video Game Research Server**

UBC PLAI Lab's research project, plaicraft.ai, aims to use AI (Artificial Intelligence) to train AGI(Artificial General Intelligence)-like agents that responds sensibly in video and audio perceptual environments in minecraft. A free, cloud and browser based minecraft game was created for the purpose of collecting minecraft gameplay from real players. However, numerous amount of data will be needed, which the research states that they aim to collect over 10,000 hours of multiplayer minecraft gameplay. Before expanding to a wider player base, the researchers will want to know certain characteristics of players who will be playing lots of minecraft, when they usually play, and predict the likelihood of players' continued play, to aid in their targeted recruitment efforts and resource management for the project accordingly. We are provided with two initial datasets about the players and each individual sessions for analysis.

## **Data Description:**
- Two different data set files were provided for analysis, players.csv and sessions.csv
- descriptions for each .csv files below

### **players.csv**
- This table captures the characteristics of each player by the 9 variables below
- There are 196 observations (rows) in total, each with a unique hashedEmail value and combination of other variables.
- There are 9 different variables (columns), each variable description described below:
| Variable | Vector Type | Description | Potentital Issues |
| :---: | :---: | --- | --- |
| **experience** | character | user rank their own experience in minecraft, 5 unique values: Beginner, Amateur, Regular, Veteran, Pro | User ranks themselves, each person could be biased in their ranking |
| **subscribe** | logical | if the user subscribed to PlaiCraft's email updates |
| **hashedEmail** | character | unique email identifier given to player as they signed up to play in PlaiCraft |
| **played_hours** | double | total hours the individual played |
| **name** | character | name given by PlaiCraft to the player when they signed up, each person has a unique name |
| **gender** | character | user chosen preferred gender, includes 7 unique values: Male, Female, Non-binary, Prefer not to say, Agender, Two-Spirited, Other | user chosen gender, some may lie about their gender
| **age** | double | user chosen age, ranges from 8 to 99 | User submitted age, some may lie about their age
| **individualId** | logical | possibly unique identifier given to individual, only contains N/A in this dataset, will drop this column as hashedEmail provided this information | no data for this variable
| **organizationName** | logical | possibly the organization the individual is associated with, only contains N/A in this dataset, will drop this column as no valuable information is contained to provide predictions | no data for this variable

In [35]:
players_df <- read_csv("data/players.csv")
head(players_df)

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,NA,NA
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17,NA,NA


### **sessions.csv**
- This table captures each session started and ended by each player
- Important to note that this dataset only has 125 unique hashedEmail, indicating that only 125 players actually started a session out of 196 from player.csv
- There are 1535 observations (rows) in total, each with a unique combination of variables.
- There are 5 different variables (columns), each variable description described below:
| Variable | Vector Type | Description |
| :---: | :---: | --- |
| **hashedEmail** | character | unique email identifier given to player as they signed up to play in PlaiCraft |
| **start_time** | character | start time of the session, with date/month/year hour:minute as value |
| **end_time** | character | end time of the session, with date/month/year hour:minute as value |
| **original_start_time** | double | scheduled start time (after player signed up) in UNIX timestamp format, representing the number of seconds since January 1, 1970 (UTC) |
| **original_end_time** | double | scheduled end time in UNIX timestamp format, representing the number of seconds since January 1, 1970 (UTC) |

In [32]:
sessions_df <- read_csv("data/sessions.csv")
head(sessions_df)

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [38]:
select(sessions_df, hashedEmail) |>
    unique() |>
    count()

n
<int>
125


## **Question Proprosal**
